In [1]:
import pandas as pd
import numpy as np


In [2]:
#Read Public Shareholders data
dfHolding = pd.read_csv("Public Shareholders Data.csv",encoding='utf-8')
print(dfHolding.shape)
dfHolding.head()

(1187, 8)


,manager,security,ISIN,as_of,total_shares_held,percent_shares_held,filing_date,filing
0,Amansa,BALKRISHNA INDUSTRIES LTD.-$,INE787D01026,2019-12-31,3829804,1.98,2020-01-15,Threshold Routine
1,Amansa,Cyient Limited,INE136B01020,2018-03-31,7235341,6.43,2018-07-16,Threshold Routine
2,Amansa,Cyient Limited,INE136B01020,2018-06-30,7235341,6.41,2018-07-16,Threshold Routine
3,Amansa,Cyient Limited,INE136B01020,2018-09-30,7235341,6.40,2018-11-01,Threshold Routine
4,Amansa,Cyient Limited,INE136B01020,2018-12-31,7235341,6.40,2019-01-18,Threshold Routine


In [3]:
#Read Security Pricing data
dfSecPricing = pd.read_csv("Security Pricing Data.csv",encoding='utf-8')
print(dfSecPricing.shape)
dfSecPricing.head()

(153473, 3)


,ISIN,Date,Close
0,IN9155A01020,12/31/2015,289.35
1,IN9155A01020,1/1/2016,296.60
2,IN9155A01020,1/4/2016,278.10
3,IN9155A01020,1/5/2016,278.95
4,IN9155A01020,1/6/2016,273.45


In [4]:
#Read Transaction data
dfTransaction = pd.read_csv("Transactions Data.csv",encoding='utf-8')
print(dfTransaction.shape)
dfTransaction.head()

(100, 10)


,manager,security,ISIN,transaction_date,quantity,price,transaction_amount,transaction_type,filing,filing_date
0,Amansa,Laurus Labs Ltd,INE947Q01010,3/14/2019,3300000,351.00,1.158300e+09,Buy,Bulk,3/14/2019
1,Amansa,RAMKRISHNA FORGINGS LTD.,INE399G01015,11/30/2018,1600000,515.24,8.243840e+08,Sell,Bulk,11/30/2018
2,Amansa,CEAT LTD.,INE482A01020,9/23/2016,237500,1171.00,2.781125e+08,Buy,Bulk,9/23/2016
3,Amansa,ENTERTAINMENT NETWORK (INDIA) LTD.,INE265F01028,11/10/2016,314544,730.00,2.296171e+08,Buy,Bulk,11/10/2016
4,Amansa,Shankara Building Products Ltd,INE274V01019,3/12/2018,147216,1755.00,2.583641e+08,Buy,Bulk,3/12/2018


In [5]:
#Read Benchmark MSCI data
dfBenchmark = pd.read_csv("Benchmark Pricing Data.csv",encoding='utf-8')
print(dfBenchmark.shape)
dfBenchmark.head()

(1067, 3)


,Benchmark,Date,Close
0,MSCI India,12/31/2015,33620.12460
1,MSCI India,1/1/2016,33652.99429
2,MSCI India,1/4/2016,32949.47460
3,MSCI India,1/5/2016,32887.33512
4,MSCI India,1/6/2016,32868.41528


In [6]:

# Below Method will Take Security frame and forward fill Quarter Date with close price for each of ISIN
def fillSecPriceForQLookup(df,uniqueISIN):
    dfSecPricingSlice = df.loc[df['ISIN'] == uniqueISIN].sort_values(by=['Date'])
    dfSecPricingSlice['Date'] = pd.to_datetime(dfSecPricingSlice['Date'])
    dfSecPricingSlice.set_index('Date',inplace=True)
    dfSecPricingSliceMod= dfSecPricingSlice.resample('D').ffill()
    
    return dfSecPricingSliceMod
        


In [7]:
# For a given ISIN - calculate Quarterly returns 

# Below Method will Take Security frame and calculate returns for Quarter end Date with close price for each of the ISIN
def getPeriodicReturns(df,resample,uniqueISIN=None):
    if uniqueISIN:
        dfSecPricingSlice = df.loc[df['ISIN'] == uniqueISIN]
        dfSecPricingSlice.drop(columns=['ISIN'],inplace=True)
    else:
        dfSecPricingSlice = df.copy()
        dfSecPricingSlice.drop(columns=['Benchmark'],inplace=True)
        #
    
    dfSecPricingSlice['Date']=pd.to_datetime(dfSecPricingSlice['Date'])
    
    dfSecPricingSlice.set_index('Date',inplace=True)
    
    dfOneSecReturns = dfSecPricingSlice.pct_change()
    
    if resample == 'B':
        d_Dsec= (1+dfOneSecReturns).resample('B',how='prod',kind='period')-1
    elif resample == 'Q':
        d_Dsec= (1+dfOneSecReturns).resample('Q',how='prod',kind='period')-1
    
    
    
    return d_Dsec
    


In [8]:
def calc_beta(df):
    np_array = df.values
    m = np_array[:,0] # market returns are column zero from numpy array
    s = np_array[:,1] # stock returns are column one from numpy array
    covariance = np.cov(s,m) # Calculate covariance between stock and market
    beta = covariance[0,1]/covariance[1,1]
    return round(beta,4)

In [9]:
def calculateBeta(dfSec,dfBenchmark,uniqueISIN=None):
    beta = 0.0
    marketReturn = getPeriodicReturns(dfBenchmark,'B')
    secReturn = getPeriodicReturns(dfSec,'B',uniqueISIN)
    
    secReturn.rename(columns={'Close':'SecReturn'},inplace=True)
    marketReturn.rename(columns={'Close':'Market Return'},inplace=True)
    
    dfCombine = secReturn.join(marketReturn)
    
    # Rearrange columns
    cols = dfCombine.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    dfCombine = dfCombine[cols]
    
    beta = calc_beta(dfCombine)
    return beta

    

In [10]:
# For a given ISIN - calculate Quarterly returns 

# Below Method will Take Security frame and calculate returns for Quarter end Date with close price for each of the ISIN
def getSecurityReturn(df,uniqueISIN,resample):
    dfSecPricingSlice = df.loc[df['ISIN'] == uniqueISIN]
    
    dfSecPricingSlice['Date']=pd.to_datetime(dfSecPricingSlice['Date'])
    dfSecPricingSlice.drop(columns=['ISIN'],inplace=True)
    dfSecPricingSlice.set_index('Date',inplace=True)
    
    dfOneSecReturns = dfSecPricingSlice.pct_change()
    res = pd.DataFrame()
    if resample == 'Q':
        d_Qsec= (1+dfOneSecReturns).resample('Q',how='prod',kind='period')-1
    
        d_Qsec.reset_index(inplace=True)
        d_Qsec['Date']  = pd.PeriodIndex(d_Qsec['Date'], freq='Q').end_time.strftime('%Y-%m-%d')
        d_Qsec['Date'] = pd.to_datetime(d_Qsec['Date'])
        res = d_Qsec
    return res
    


In [11]:
# Populate return for each of secrity in holding frame
# Build and Enhance dfHolding Framework with Returns
def enhanceHolding(dfHolding,dfSecPricing,dfBenchmark):
    uniqueISINs = list(set(dfHolding.ISIN.values))
    dfHolding.reset_index(drop=True,inplace=True)
    frameList = []
    beta = 0.0
    for isin in uniqueISINs:
        
        # Beta calculation
        beta = calculateBeta(dfSecPricing,dfBenchmark,isin)
        
        secFrame = fillSecPriceForQLookup(dfSecPricing,isin)
        secFrame.reset_index(inplace=True)
        dfHoldingSlice = dfHolding.loc[dfHolding['ISIN'] == isin]
        dfHoldingSlice.reset_index(drop=True,inplace=True)
        
        # Get Price return for Each of ISIN Security Price
        returnFrame = getSecurityReturn(dfSecPricing,isin,'Q')
        
        for i in range(len(dfHoldingSlice.index)):
            targetDate = str(dfHoldingSlice.loc[i,'as_of'])
            totalSharesHeld = dfHoldingSlice.loc[i,'total_shares_held']
            
            frame = secFrame.loc[(secFrame['ISIN'] == isin) & (pd.to_datetime(secFrame['Date']) == pd.to_datetime(targetDate))]
            if frame.shape[0] > 0:
                val = frame.Close.values[0]
            else:
                val = 0.0
                
            # Return Frame
            returns = returnFrame.loc[pd.to_datetime(returnFrame['Date']) == pd.to_datetime(targetDate)]
            if returns.shape[0] > 0:
                secReturn = returns.Close.values[0]
            else:
                secReturn = 0.0
            
            
            dfHoldingSlice.loc[i,'Close'] = val
            dfHoldingSlice.loc[i,'MKTValue'] = 1.0*val*totalSharesHeld
            
            dfHoldingSlice.loc[i,'PriceReturn'] = secReturn
            dfHoldingSlice.loc[i,'beta'] = beta
        frameList.append(dfHoldingSlice)
    dfRaw = pd.concat(frameList)
    dfRaw['MKTValue'] =round(pd.to_numeric(dfRaw['MKTValue']),2)
    
    dfRes = dfRaw.sort_values(by=['manager','ISIN'])
    dfRes.reset_index(drop=True,inplace=True)
    return dfRes
            
    


In [12]:
# Needed to fetch closing price
def getClosingPrice(dfSecPricing,isin,txDate):
    secFrame = fillSecPriceForQLookup(dfSecPricing,isin)
    secFrame.reset_index(inplace=True)
    frame = secFrame.loc[(secFrame['ISIN'] == isin) & 
                                     (pd.to_datetime(secFrame['Date']) == pd.to_datetime(txDate))]
    val = 0.0
    if frame.shape[0] > 0:
        val = frame.Close.values[0]
    return val


In [13]:
# Ofsetting adjustments for Market Value calculation
def findOffSetQtr(givenDate,shift):
    convDate = pd.to_datetime(givenDate)
    res = convDate - pd.tseries.offsets.QuarterEnd(n=shift)
    return res

In [14]:
# Get Enhanced Holding frame with Closing Price and MKTValue
dfHoldingWithCP = enhanceHolding(dfHolding,dfSecPricing,dfBenchmark)
print(dfHoldingWithCP.shape)
dfHoldingWithCP.head()

C:\Users\kkcar\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).prod()
C:\Users\kkcar\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\kkcar\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\kkcar\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

(1187, 12)


,manager,security,ISIN,as_of,total_shares_held,percent_shares_held,filing_date,filing,Close,MKTValue,PriceReturn,beta
0,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2018-06-30,14303747,2.81,2018-07-21,Threshold Routine,158.65,2.269289e+09,-0.134479,1.511
1,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-09-30,8069750,1.59,2017-10-10,Threshold Routine,222.30,1.793905e+09,-0.155716,1.511
2,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-06-30,9000000,1.77,2017-07-21,Threshold Routine,263.30,2.369700e+09,-0.066808,1.511
3,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-03-31,9603371,1.89,2017-04-17,Threshold Routine,282.15,2.709591e+09,-0.056670,1.511
4,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2016-12-31,7300000,1.44,2017-01-20,Threshold Routine,299.10,2.183430e+09,-0.121586,1.511


In [15]:
#dfHoldingWithCP.to_csv("dfHoldingWithCP.csv",index=False,encoding='utf-8')

# Transaction block

In [16]:
dfTransaction['transaction_date'] = pd.to_datetime(dfTransaction['transaction_date'])
dfTransaction['QuarterEndDate'] = dfTransaction['transaction_date'] + pd.tseries.offsets.QuarterEnd(0)
print(dfTransaction.shape)
dfTransaction.head()

(100, 11)


,manager,security,ISIN,transaction_date,quantity,price,transaction_amount,transaction_type,filing,filing_date,QuarterEndDate
0,Amansa,Laurus Labs Ltd,INE947Q01010,2019-03-14,3300000,351.00,1.158300e+09,Buy,Bulk,3/14/2019,2019-03-31
1,Amansa,RAMKRISHNA FORGINGS LTD.,INE399G01015,2018-11-30,1600000,515.24,8.243840e+08,Sell,Bulk,11/30/2018,2018-12-31
2,Amansa,CEAT LTD.,INE482A01020,2016-09-23,237500,1171.00,2.781125e+08,Buy,Bulk,9/23/2016,2016-09-30
3,Amansa,ENTERTAINMENT NETWORK (INDIA) LTD.,INE265F01028,2016-11-10,314544,730.00,2.296171e+08,Buy,Bulk,11/10/2016,2016-12-31
4,Amansa,Shankara Building Products Ltd,INE274V01019,2018-03-12,147216,1755.00,2.583641e+08,Buy,Bulk,3/12/2018,2018-03-31


In [17]:
def enrichHoldingWithTx(dfHoldingWithCP,dfTransaction,dfSecPricing):
    resultList = []
    nonReportingCount = 0
    for i in range(len(dfTransaction.index)):
        rowList = []
        txManager = dfTransaction.loc[i,'manager']
        txSecurity = dfTransaction.loc[i,'security']
        txISIN = dfTransaction.loc[i,'ISIN']
        txQty = dfTransaction.loc[i,'quantity']
        txPrice = dfTransaction.loc[i,'price']
        txMktVal = dfTransaction.loc[i,'transaction_amount']
        txSignal = dfTransaction.loc[i,'transaction_type']
        txFiling = dfTransaction.loc[i,'filing']
        txFilingDate = dfTransaction.loc[i,'filing_date']
        txQEnd = dfTransaction.loc[i,'QuarterEndDate']
        
        dfHold = (dfHoldingWithCP['manager'] == txManager) & (dfHoldingWithCP['ISIN'] ==txISIN )
        dfHoldTotal = dfHoldingWithCP.loc[dfHold]
        dfHoldTotal.reset_index(drop=True,inplace=True)
        if dfHoldTotal.shape[0] > 0:
            ts = dfHoldTotal.loc[0,'total_shares_held']
            tp = dfHoldTotal.loc[0,'percent_shares_held']
            totalQty = (100/tp)*ts
        
        dfHoldFlag = (dfHoldingWithCP['manager'] == txManager) & (dfHoldingWithCP['ISIN'] ==txISIN ) & (pd.to_datetime(dfHoldingWithCP['as_of']) == pd.to_datetime(txQEnd)) 
        dfHoldTest = dfHoldingWithCP.loc[dfHoldFlag]
        
        prevQtr = findOffSetQtr(txQEnd,1)
        prevQtrFlag = (dfHoldingWithCP['manager'] == txManager) & (dfHoldingWithCP['ISIN'] ==txISIN ) & (pd.to_datetime(dfHoldingWithCP['as_of']) == pd.to_datetime(prevQtr)) 
        prevQtrTest = dfHoldingWithCP.loc[prevQtrFlag]
        
        if dfHoldTest.shape[0] > 0:
            #print("For Manager {} , ISIN {} , QEDate {} and Signal {} found".format(txManager,txISIN,txQEnd,txSignal))
             
            #CASE 1 FOUND AND BUY 
            if dfHoldTest.PriceReturn.values[0]:
                priceVal = dfHoldTest.PriceReturn.values[0]
            else:
                priceVal = 0.0
                
            if dfHoldTest.beta.values[0]:
                beta = dfHoldTest.beta.values[0]
            else:
                beta = 0.0
            
            
                
            if txSignal == "Buy":
                #print("CASE1 BUY BLOCK")
                rowList.append(txManager)
                rowList.append(txSecurity)
                rowList.append(txISIN)
                
                rowList.append(txQEnd)
                rowList.append(dfHoldTest.total_shares_held.values[0])
                rowList.append(dfHoldTest.percent_shares_held.values[0])
                rowList.append(dfHoldTest.filing_date.values[0])
                rowList.append(dfHoldTest.filing.values[0])
                rowList.append(dfHoldTest.Close.values[0])
            
                
                adjustedMktVal = dfHoldTest.MKTValue.values[0] - dfHoldTest.total_shares_held.values[0] * dfHoldTest.Close.values[0] + txMktVal
                rowList.append(adjustedMktVal)
                #
                rowList.append(priceVal)
                rowList.append(beta)
            # Case 2 Found and Sell
            elif txSignal == "Sell":
                # No need to adjust marketvalue - Reporting might be done correctly
                #print("CASE2 SELL BLOCK")
                rowList.append(txManager)
                rowList.append(txSecurity)
                rowList.append(txISIN)
                
                rowList.append(txQEnd)
                rowList.append(txQty)
                rowList.append(dfHoldTest.percent_shares_held.values[0])
                rowList.append(dfHoldTest.filing_date.values[0])
                rowList.append(dfHoldTest.filing.values[0])
                rowList.append(dfHoldTest.Close.values[0])
                rowList.append(dfHoldTest.MKTValue.values[0])
                rowList.append(priceVal)
                rowList.append(beta)
                
        elif dfHoldTest.shape[0] == 0:
            # Case 3 and Case 4 -  copy and establish position
            #print("For Manager {} , ISIN {} , QEDate {} and Signal {} NOT found".format(txManager,txISIN,txQEnd,txSignal))
            # Case 3 - Not Found AND Buy
            if txSignal == "Buy":
                #print("CASE3 BUY BLOCK")
                rowList.append(txManager)
                rowList.append(txSecurity)
                rowList.append(txISIN)
        
                rowList.append(txQEnd)
                if prevQtrTest.shape[0] == 0:
                    rowList.append(txQty)
                    os = round((txQty/totalQty)*100,2)
                    rowList.append(os)
                    rowList.append(txFilingDate)
                    rowList.append(txFiling)
                    
                    # Assuming -it is filing date price - not updating with close price at qtr end
                    closePrice = getClosingPrice(dfSecPricing,txISIN,txQEnd)
                    #rowList.append(txPrice)
                    rowList.append(closePrice)
                    rowList.append(txMktVal)
                    rowList.append(priceVal)
                    rowList.append(beta)
                else:
                    # Assuming -it is filing date price - not updating with close price at qtr end
                    closePrice = getClosingPrice(dfSecPricing,txISIN,txQEnd)
                                        
                    # Assumption
                    updatedQty = txQty+prevQtrTest.total_shares_held.values[0]
                    rowList.append(updatedQty)
                    os = round((updatedQty/totalQty)*100,2)
                    rowList.append(os)
                    rowList.append(txFilingDate)
                    rowList.append(txFiling)
                    
                    rowList.append(closePrice)
                    updatedMarketVal = txMktVal + prevQtrTest.MKTValue.values[0]
                    rowList.append(updatedMarketVal)
                    rowList.append(priceVal)
                    rowList.append(beta)
            
            if txSignal == "Sell":
                
                if prevQtrTest.shape[0] == 0:
                    # Assuming they do not have any position to report and hence not reporting any line item
                    nonReportingCount += 1
                    #print("CASE4 PREVIOUS QTR NON REPORTING SELL BLOCK with running count {}".format(nonReportingCount))
                    
                else:
                    # Assuming -it is filing date price - not updating with close price at qtr end
                    
                    closePrice = getClosingPrice(dfSecPricing,txISIN,txQEnd)
                    
                    updatedMarketVal =   prevQtrTest.MKTValue.values[0] - txMktVal
                    
                    if updatedMarketVal > 0.0:
                        rowList.append(txManager)
                        rowList.append(txSecurity)
                        rowList.append(txISIN)
                        rowList.append(txQEnd)
                    
                        
                        # Assumption - Even if QTY goes zero - but MKTVAL remais positive - we are reporting the block
                        updatedQty = prevQtrTest.total_shares_held.values[0] - txQty
                        rowList.append(updatedQty)
                        #print("CASE4 BLOCK with updatedQty {} and updatedMKTVAL {}".format(updatedQty,updatedMarketVal))
                        os = round((updatedQty/totalQty)*100,2)
                        rowList.append(os)
                        rowList.append(txFilingDate)
                        rowList.append(txFiling)
                        
                        rowList.append(closePrice)
                    
                        rowList.append(updatedMarketVal)
                        rowList.append(priceVal)
                        rowList.append(beta)
                    else:
                        nonReportingCount += 1
                        #print("CASE4 NON REPORTING BLOCK with running count {}".format(nonReportingCount))
                        # All positions are exhausted and hence noting to report
        if len(rowList) > 0:
            resultList.append(rowList)
    return resultList


In [18]:
# Get Adjusted Transactions
resultList = enrichHoldingWithTx(dfHoldingWithCP,dfTransaction,dfSecPricing)
print(len(resultList))

70


In [20]:
# Concatenate it with Holding Framework
colList = dfHoldingWithCP.columns.tolist()

dfTransactionUpdated = pd.DataFrame(resultList,columns=colList)
assert dfTransactionUpdated.shape[0] == 70

#dfTransactionUpdated.to_csv("dfTransactionUpdatedFrame.csv",index=False,encoding='utf-8')

dfHoldingWithTX=pd.concat([dfHoldingWithCP,dfTransactionUpdated])
assert dfHoldingWithTX.shape[0] == 1257

In [21]:
print(dfHoldingWithTX.shape)
dfHoldingWithTX.head()

(1257, 12)


,manager,security,ISIN,as_of,total_shares_held,percent_shares_held,filing_date,filing,Close,MKTValue,PriceReturn,beta
0,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2018-06-30,14303747,2.81,2018-07-21,Threshold Routine,158.65,2.269289e+09,-0.134479,1.511
1,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-09-30,8069750,1.59,2017-10-10,Threshold Routine,222.30,1.793905e+09,-0.155716,1.511
2,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-06-30,9000000,1.77,2017-07-21,Threshold Routine,263.30,2.369700e+09,-0.066808,1.511
3,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-03-31,9603371,1.89,2017-04-17,Threshold Routine,282.15,2.709591e+09,-0.056670,1.511
4,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2016-12-31,7300000,1.44,2017-01-20,Threshold Routine,299.10,2.183430e+09,-0.121586,1.511


In [22]:
#dfHoldingWithTX.to_csv("dfHoldingWithTX.csv",index=False,encoding='utf-8')

# Enhance Holding Framework with Portfolio Weights

In [23]:
# For Each of "as_of" date - calculate Total Market Value - in a seperate frame


In [24]:
dfHoldingWithTX['as_of'] = pd.to_datetime(dfHoldingWithTX['as_of'])

In [25]:
df_agg = dfHoldingWithTX.groupby(['manager', 'as_of']).agg({'MKTValue': ['sum']})
df_agg.columns = ['MKTValueTotal']
df_agg = df_agg.reset_index()
print(df_agg.shape)
df_agg.head()

(51, 3)


,manager,as_of,MKTValueTotal
0,Amansa,2015-12-31,2.458036e+10
1,Amansa,2016-03-31,2.850678e+09
2,Amansa,2016-06-30,4.197586e+10
3,Amansa,2016-09-30,5.454055e+10
4,Amansa,2016-12-31,5.708603e+10


In [26]:
#df_agg.to_csv("MKTValue.csv",index=False,encoding='utf-8')

In [27]:
# Now we have aggrgated market frame - populate dfHoldingWithTX with Total Market Value and Portfolio weights
def enhancePortfolioWeight(dfHoldingWithTXCopy,df_agg):
    dfHoldingWithTXCopy.reset_index(drop=True,inplace=True)
    for i in range(len(dfHoldingWithTXCopy.index)):
    
        holdManager = dfHoldingWithTXCopy.loc[i,'manager']
        holdAsOf    = dfHoldingWithTXCopy.loc[i,'as_of']
        holdmktValue = dfHoldingWithTXCopy.loc[i,'MKTValue']
        #print(" hold : Manager {} , AsOf {} ,and Mktvalue {}".format(holdManager,holdAsOf,holdmktValue))
        # get Total market value
        dfHoldFlag = (df_agg['manager'] == holdManager) & (pd.to_datetime(df_agg['as_of']) == pd.to_datetime(holdAsOf)) 
        
        dfHoldTest = df_agg.loc[dfHoldFlag]
        
        value = 0.0
        if dfHoldTest.shape[0] > 0:
            value = dfHoldTest.MKTValueTotal.values[0]
            dfHoldingWithTXCopy.loc[i,'TotalMktValue'] = value
            dfHoldingWithTXCopy.loc[i,'PWeight'] = round((holdmktValue/value)*100,4)
    return dfHoldingWithTXCopy
    
    
        


In [28]:
dfHoldingWithWeights = enhancePortfolioWeight(dfHoldingWithTX,df_agg)


In [29]:
print(dfHoldingWithWeights.shape)
dfHoldingWithWeights.head()

(1257, 14)


,manager,security,ISIN,as_of,total_shares_held,percent_shares_held,filing_date,filing,Close,MKTValue,PriceReturn,beta,TotalMktValue,PWeight
0,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2018-06-30,14303747,2.81,2018-07-21,Threshold Routine,158.65,2.269289e+09,-0.134479,1.511,8.019274e+10,2.8298
1,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-09-30,8069750,1.59,2017-10-10,Threshold Routine,222.30,1.793905e+09,-0.155716,1.511,7.276921e+10,2.4652
2,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-06-30,9000000,1.77,2017-07-21,Threshold Routine,263.30,2.369700e+09,-0.066808,1.511,6.657460e+10,3.5595
3,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-03-31,9603371,1.89,2017-04-17,Threshold Routine,282.15,2.709591e+09,-0.056670,1.511,6.853669e+10,3.9535
4,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2016-12-31,7300000,1.44,2017-01-20,Threshold Routine,299.10,2.183430e+09,-0.121586,1.511,5.708603e+10,3.8248


In [30]:
dfHoldingWithWeights['SecReturn'] = round(dfHoldingWithWeights['PriceReturn'] * dfHoldingWithWeights['PWeight']*0.01,6)
dfHoldingWithWeights['WeightedBeta'] = round(dfHoldingWithWeights['beta'] * dfHoldingWithWeights['PWeight']*0.01,6)
print(dfHoldingWithWeights.shape)
dfHoldingWithWeights.head()

(1257, 16)


,manager,security,ISIN,as_of,total_shares_held,percent_shares_held,filing_date,filing,Close,MKTValue,PriceReturn,beta,TotalMktValue,PWeight,SecReturn,WeightedBeta
0,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2018-06-30,14303747,2.81,2018-07-21,Threshold Routine,158.65,2.269289e+09,-0.134479,1.511,8.019274e+10,2.8298,-0.003805,0.042758
1,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-09-30,8069750,1.59,2017-10-10,Threshold Routine,222.30,1.793905e+09,-0.155716,1.511,7.276921e+10,2.4652,-0.003839,0.037249
2,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-06-30,9000000,1.77,2017-07-21,Threshold Routine,263.30,2.369700e+09,-0.066808,1.511,6.657460e+10,3.5595,-0.002378,0.053784
3,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2017-03-31,9603371,1.89,2017-04-17,Threshold Routine,282.15,2.709591e+09,-0.056670,1.511,6.853669e+10,3.9535,-0.002240,0.059737
4,Amansa,Tata Motors Ltd - DVR,IN9155A01020,2016-12-31,7300000,1.44,2017-01-20,Threshold Routine,299.10,2.183430e+09,-0.121586,1.511,5.708603e+10,3.8248,-0.004650,0.057793


In [32]:
dfHoldingWithWeights.to_csv("dfPortfolioHoldingWithWeights.csv",index=False,encoding='utf-8')


# Analytics

In [33]:
def getCombinedPortfolio(df):
    dfPortfolio = df.groupby(['manager', 'as_of','TotalMktValue']).agg({'SecReturn': ['sum'],'WeightedBeta': ['sum']})
    dfPortfolio.columns = ['PortfolioReturn','PortfolioBeta']
    dfPortfolio = dfPortfolio.reset_index()
    dfPortfolio.rename(columns={"manager":"Manager","as_of":"QuarterEnd"},inplace=True)
    dfPortfolio['PortfolioReturnPercentage'] = round(dfPortfolio['PortfolioReturn']*100,4)
    dfPortfolio['PortfolioBeta'] = round(dfPortfolio['PortfolioBeta']*1.0,4)
    return dfPortfolio

    

In [34]:
def getBenchmarkQuaterlyReturns(dfBenchmark):
    dfBenchmarkReturn = getPeriodicReturns(dfBenchmark,'Q')
    dfBenchmarkReturn.reset_index(inplace=True)
    dfBenchmarkReturn['Date']  = pd.PeriodIndex(dfBenchmarkReturn['Date'], freq='Q').end_time.strftime('%Y-%m-%d')
    dfBenchmarkReturn['Date'] = pd.to_datetime(dfBenchmarkReturn['Date'])
    return dfBenchmarkReturn
        


In [35]:
def enhancePortfolio(dfPortfolio,dfBenchmarkReturn):
    for i in range(len(dfPortfolio.index)):
        qtrDate = dfPortfolio.loc[i,'QuarterEnd']

        benchFlag = pd.to_datetime(dfBenchmarkReturn['Date']) == pd.to_datetime(qtrDate) 
        benchFrame = dfBenchmarkReturn.loc[benchFlag]



        if benchFrame.shape[0] > 0:
            mktRet = benchFrame.Close.values[0]
        else:
            mktRet = 0
        dfPortfolio.loc[i,'MktReturn'] = mktRet
    return dfPortfolio
    
    

In [36]:
# rolling returns
def getRollingAndExcessReturn(dfPortfolio):
    
    dfPortfolio.set_index('QuarterEnd',inplace=True)
    dfPortfolio['rolling_6Months']=dfPortfolio.PortfolioReturn.rolling(2).mean()
    dfPortfolio['BMExcessRt'] = round(dfPortfolio['PortfolioReturn']- dfPortfolio['MktReturn'],4)
    # This is Alpha
    dfPortfolio['alpha'] = round((dfPortfolio['PortfolioReturn'] -dfPortfolio['PortfolioBeta']*dfPortfolio['MktReturn']),4)
    dfPortfolio.reset_index(inplace=True)
    return dfPortfolio


In [37]:
def getBenchmarkReturn(dfPortfolio):
    mgrSlice = dfPortfolio.loc[dfPortfolio['Manager'] == 'Amansa']
    benchreturnSeries = list(mgrSlice.MktReturn.values)
    breturn = round(np.prod([(1+x) for x in benchreturnSeries])-1.0,4)
    return breturn
        
    

In [38]:
def getAnalytics(df):
    resList = []
    uniqueManagers = list(set(df.Manager.values))
    breturn = round(getBenchmarkReturn(df),4)
    
    for mgr in uniqueManagers:
        rowList = []
        mgrSlice = df.loc[df['Manager'] == mgr]
        rowList.append(mgr)
        returnSeries = list(mgrSlice.PortfolioReturn.values)
        qreturn = round(np.prod([(1+x) for x in returnSeries])-1.0,4)
        variance = round(mgrSlice['BMExcessRt'].var(),4)
        sdev = round(mgrSlice['BMExcessRt'].std(),4)
        
        riskAdjusted = round((qreturn-breturn)/sdev,4)
        
        betaSeries  = list(mgrSlice.PortfolioBeta.values)
        bMExcessRtSeries = list(mgrSlice.BMExcessRt.values)
        
        pbeta = round(sum(betaSeries)/len(betaSeries),4)
        
        bexcess = round((qreturn-breturn),4)
        
        alpha = round((qreturn -pbeta*breturn),4) 
        
        
        rowList.append(qreturn)
        rowList.append(variance)
        rowList.append(sdev)
        rowList.append(breturn)
        rowList.append(riskAdjusted)
        
        rowList.append(pbeta)
        rowList.append(bexcess)
        rowList.append(alpha)
        
        
        if len(rowList) > 0:
            resList.append(rowList)
    
    return resList
        

In [39]:
dfPortDetails = getCombinedPortfolio(dfHoldingWithWeights)
dfBenchmarkReturn = getBenchmarkQuaterlyReturns(dfBenchmark)
dfPortDetailsEnhanced = enhancePortfolio(dfPortDetails,dfBenchmarkReturn)
dfPortfolio = getRollingAndExcessReturn(dfPortDetailsEnhanced)
print(dfPortfolio.shape)
dfPortfolio.head()



(51, 10)


C:\Users\kkcar\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).prod()


,QuarterEnd,Manager,TotalMktValue,PortfolioReturn,PortfolioBeta,PortfolioReturnPercentage,MktReturn,rolling_6Months,BMExcessRt,alpha
0,2015-12-31,Amansa,2.458036e+10,0.000000,0.7379,0.0000,0.000000,NaN,0.0000,0.0000
1,2016-03-31,Amansa,2.850678e+09,-0.112496,0.7408,-11.2496,-0.026230,-0.056248,-0.0863,-0.0931
2,2016-06-30,Amansa,4.197586e+10,0.154192,0.8662,15.4192,0.058880,0.020848,0.0953,0.1032
3,2016-09-30,Amansa,5.454055e+10,0.159724,0.8960,15.9724,0.045125,0.156958,0.1146,0.1193
4,2016-12-31,Amansa,5.708603e+10,-0.064564,0.8850,-6.4564,-0.061264,0.047580,-0.0033,-0.0103


In [40]:
dfPortfolio.to_csv("dfPortfolioSummary.csv",index=False,encoding='utf-8')


In [41]:
statsList = getAnalytics(dfPortfolio)
statsList

[['Steadview',
  2.5374,
  0.0189,
  0.1375,
  0.4666,
  15.0604,
  0.9656,
  2.0708,
  2.0869],
 ['Elara India',
  1.6602,
  0.026,
  0.1612,
  0.4666,
  7.4045,
  1.1215,
  1.1936,
  1.1369],
 ['Amansa', 0.9389, 0.0031, 0.0557, 0.4666, 8.4794, 0.8361, 0.4723, 0.5488]]

In [42]:
colList = ['Manager','TotalReturn','Variance','Std. Deviation','TotalMarketReturn','InformationRatio','AvgBeta','ExcessReturn','PluginAlpha']
dfStats = pd.DataFrame(statsList,columns=colList)
print(dfStats.shape)
dfStats.head()

(3, 9)


,Manager,TotalReturn,Variance,Std. Deviation,TotalMarketReturn,InformationRatio,AvgBeta,ExcessReturn,PluginAlpha
0,Steadview,2.5374,0.0189,0.1375,0.4666,15.0604,0.9656,2.0708,2.0869
1,Elara India,1.6602,0.0260,0.1612,0.4666,7.4045,1.1215,1.1936,1.1369
2,Amansa,0.9389,0.0031,0.0557,0.4666,8.4794,0.8361,0.4723,0.5488


In [43]:
#dfStats.to_csv("dfPortfolioStats.csv",index=False,encoding='utf-8')

In [44]:
dfStats['Rank'] = dfStats["InformationRatio"].rank(method ='max')
dfStats.sort_values(by=['Rank'],inplace=True)
print(dfStats.shape)
dfStats.head()

(3, 10)


,Manager,TotalReturn,Variance,Std. Deviation,TotalMarketReturn,InformationRatio,AvgBeta,ExcessReturn,PluginAlpha,Rank
1,Elara India,1.6602,0.0260,0.1612,0.4666,7.4045,1.1215,1.1936,1.1369,1.0
2,Amansa,0.9389,0.0031,0.0557,0.4666,8.4794,0.8361,0.4723,0.5488,2.0
0,Steadview,2.5374,0.0189,0.1375,0.4666,15.0604,0.9656,2.0708,2.0869,3.0


In [45]:
dfStats.to_csv("dfPortfolioRankWithRiskAdjustedRatio.csv",index=False,encoding='utf-8')

In [46]:
dfStats['Rank'] = dfStats["TotalReturn"].rank(method ='max')
dfStats.sort_values(by=['Rank'],inplace=True)
print(dfStats.shape)
dfStats.head()

(3, 10)


,Manager,TotalReturn,Variance,Std. Deviation,TotalMarketReturn,InformationRatio,AvgBeta,ExcessReturn,PluginAlpha,Rank
2,Amansa,0.9389,0.0031,0.0557,0.4666,8.4794,0.8361,0.4723,0.5488,1.0
1,Elara India,1.6602,0.0260,0.1612,0.4666,7.4045,1.1215,1.1936,1.1369,2.0
0,Steadview,2.5374,0.0189,0.1375,0.4666,15.0604,0.9656,2.0708,2.0869,3.0


In [47]:
dfStats.to_csv("dfPortfolioRankWithTotalReturn.csv",index=False,encoding='utf-8')